Conduct RFM (recency, frequency, monetary value) analysis on political donors in the UK

In [1]:
#imports
import pandas as pd
import datetime as dt 
import numpy as np

# DATA PROCESSING

In [2]:
#read data
df = pd.read_csv('data/Donations accepted by political parties.csv')
df.shape

/var/folders/9t/ywq9gg3s4zbgn2zw5tfskbv00000gn/T/ipykernel_23597/3107819911.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Donations accepted by political parties.csv')


(65278, 29)

In [3]:

#drop all nans from the columns that we will be using
df = df.dropna(subset=['Value', 'DonorId', 'ReceivedDate'])
df.shape

(62492, 29)

In [4]:
#change received data from string to datetime
df['ReceivedDate'] = pd.to_datetime(df['ReceivedDate'], format='%d/%m/%Y')


# RFM ANALYSIS 

In [5]:
#Compute recency values
df_recency = df.groupby(by='DonorId', as_index=False)['ReceivedDate'].max()
df_recency.columns = ['DonorId', 'LastPurchaseDate']
recent_date = df_recency['LastPurchaseDate'].max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(
    lambda x: (recent_date - x).days)
df_recency.head

<bound method NDFrame.head of        DonorId LastPurchaseDate  Recency
0          1.0       2001-05-14     6685
1          2.0       2003-08-11     5866
2          3.0       2015-02-18     1657
3          4.0       2010-12-31     3167
4          5.0       2009-12-15     3548
...        ...              ...      ...
42183  87473.0       2019-08-05       28
42184  87478.0       2018-11-01      305
42185  87479.0       2009-12-29     3534
42186  87480.0       2017-03-13      903
42187  87482.0       2014-02-06     2034

[42188 rows x 3 columns]>

In [ ]:
#Calculate Frequency
